## Pre-requisite Python Packages
Please add these using the "Packages" drop-down:
  - plotly=6.0.1
  - snowflake=1.6.0
  - snowflake-snowpark-python=1.35.0
  - snowflake.core=1.6.0

## Using Technical Indicators for Historical Analysis
- ### Validate Investment Theses: Use past data to confirm if a trading strategy would have worked.
- ### Identify Historical Trends: Discover long-term patterns of accumulation or distribution.
- ### Analyze Market Conviction: Understand the strength behind historical price movements.
- ### Optimize Portfolio Decisions: Refine entry and exit points for future trades based on past performance.

### The Challenge: Scaling Technical Analysis

While essential, calculating dozens of indicators across years of historical data presents significant data engineering challenges that can slow down or block analysis entirely.

**Typical Hurdles:**
* **Massive Data Wrangling:** Sourcing, cleaning, and aligning terabytes of granular trade and quote data is a complex and time-consuming first step.
* **Complex Coding & Validation:** Each indicator—like RSI, MACD, or Bollinger Bands—has a unique mathematical formula that must be coded, tested, and validated.
* **Intensive Computation:** The window functions and aggregations required are computationally expensive, often leading to slow performance on traditional systems.
* **Brittle Pipelines:** Managing complex ETL jobs to chain these calculations together is an enormous engineering burden. If one step fails, the entire analysis breaks.

**The Snowflake Solution:**
* **Simplified Pipelines with Dynamic Tables:** We can replace complex, procedural ETL with simple, declarative SQL, letting Snowflake automate the dependencies and refreshes.
* **Powerful SQL Engine:** Snowflake's engine is built for the scale and complexity of financial analysis, handling massive window functions and aggregations with ease.
* **Separation of Compute & Storage:** We can instantly scale a warehouse up for heavy calculations and then scale it down, paying only for the compute we use.
* **Snowpark for Python:** For highly specialized indicators, we can bring Python's rich libraries directly to the data, avoiding data movement.

# Technical Indicators with Snowflake Dynamic Tables
## Volume Weighted Average Price (VWAP) Analysis

This notebook demonstrates how to calculate technical indicators using **Snowflake's Dynamic Tables** feature. We'll focus on Volume Weighted Average Price (VWAP) calculation using realistic stock market data.

### Key Features:
- **Realistic Market Data**: Generated using actual market patterns from AAPL and other major stocks
- **Dynamic Tables**: Automated refresh with incremental processing
- **Time Series Analysis**: 20-minute time slices and cumulative calculations
- **Interactive Visualization**: Streamlit charts comparing intermediate vs final VWAP

### Architecture Overview:
1. **Raw Data Ingestion**: JSON market data simulation
2. **Data Normalization**: Flatten JSON into structured records
3. **Time Slice Aggregation**: 20-minute VWAP calculations
4. **Cumulative Analysis**: Running VWAP using window functions
5. **Visualization**: Interactive Streamlit dashboard


## What Are Dynamic Tables? The Future of Data Pipelines

Think of Dynamic Tables as **automated and declarative data pipelines** built directly into Snowflake. Instead of writing complex, multi-step procedures with streams and tasks, you simply declare the final state of your data using a single `SELECT` statement.

### Key Benefits for This Demo:

* **Declarative & Simple**: You define **what** you want the final data to look like, and Snowflake handles **how** to get it there. Notice we don't write any procedural code to manage refreshes.

* **Automated Orchestration**: Snowflake automatically builds and manages the entire dependency graph (DAG) between tables. When raw data is updated, Snowflake refreshes every downstream table in the correct order.

* **Controlled Data Freshness**: You can easily define how up-to-date your data needs to be using the `TARGET_LAG` parameter, from near real-time to hours or days.

* **Efficient & Cost-Effective**: Refreshes are incremental by default, only processing new or changed data. This runs on serverless compute, optimizing performance and cost.

## Step 1: Environment Setup

Set up the Snowflake environment with database, schema, and warehouse.


In [ ]:
-- Environment Setup
USE DATABASE DEMODB;
USE SCHEMA EQUITY_RESEARCH;
USE WAREHOUSE DEMO_XSMALL_WH;

-- Verify connection
SELECT CURRENT_DATABASE(), CURRENT_SCHEMA(), CURRENT_WAREHOUSE();


## Step 2: Create Raw Data Ingestion Table

This transient table simulates the landing zone for JSON market data from external sources.


In [ ]:
-- Create raw data ingestion table
CREATE OR REPLACE TRANSIENT TABLE DEMODB.EQUITY_RESEARCH.DEMO_MARKET_DATA_JSON_INGESTION_TTBL (
    TICKER VARIANT,
    RESULTS VARIANT
)
COMMENT = 'Raw JSON market data ingestion table - simulates data loading from an external stage using Snowpipe';


## Step 3: Create Python Stored Procedure for Realistic AAPL Data

Let's insert simulated trade data for 9 popular stocks. Each record contains:
- **c**: Close price
- **h**: High price  
- **l**: Low price
- **n**: Number of trades
- **o**: Open price
- **t**: Timestamp (Unix milliseconds)
- **v**: Volume
- **vw**: Volume weighted price

This Python stored procedure generates realistic AAPL stock data using advanced statistical modeling with numpy. It creates data that closely mimics real market patterns including:

- **Geometric Brownian Motion**: For realistic price movements
- **Intraday Volume Patterns**: Higher volume at open/close, lower at midday
- **Proper OHLC Relationships**: Mathematically consistent price data
- **Market Hours Only**: Trading data during 9:30 AM - 4:00 PM EST, weekdays only


## Format of the JSON Data Loaded into the raw table  
```
[
  {
    "c": 144.01,
    "h": 144.29,
    "l": 144.01,
    "n": 68,
    "o": 144.29,
    "t": 1675242000000,
    "v": 2665,
    "vw": 144.1408
  },
  {
    "c": 144.09,
    "h": 144.1,
    "l": 144.09,
    "n": 36,
    "o": 144.09,
    "t": 1675242060000,
    "v": 1174,
    "vw": 144.0971
  }
]

## Generate Realistic Stock Trades Data

In [ ]:
CREATE OR REPLACE PROCEDURE DEMODB.EQUITY_RESEARCH.GENERATE_AAPL_DATA_SIMULATION_PY_FINAL(
    START_DATE_STR VARCHAR,
    END_DATE_STR VARCHAR
)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = '3.11'
PACKAGES = ('snowflake-snowpark-python', 'numpy')
HANDLER = 'run'
AS
$$
import json
from datetime import datetime, timedelta
import numpy as np

def run(session, start_date_str, end_date_str):
    # ===================================================================
    # 1. Configuration Parameters
    #    These variables control the simulation's behavior and are based
    #    on realistic historical data for a stock like AAPL.
    # ===================================================================
    start_price = 130.0         # The price at which the simulation begins.
    end_price = 230.0           # The target price at the end of the simulation period.
    daily_volatility = 0.025    # The expected daily price fluctuation (e.g., 2.5%).
    base_volume_per_min = 150000 # The average trading volume per minute.
    ticker = 'AAPL'             # The stock ticker symbol.

    # Convert the input strings to datetime objects for calculations.
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d').date()
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d').date()

    # ===================================================================
    # 2. Procedural Loop in Python to Generate Data Points
    #    This section generates the minute-by-minute trade data.
    # ===================================================================
    all_datapoints = []         # An empty list to store the generated trade data.
    current_date = start_date   # The date for the current iteration of the loop.
    last_close_price = start_price # The closing price of the previous minute.

    # Pre-calculate the drift and volatility to use in the loop.
    total_days = (end_date - start_date).days
    minute_drift = (pow(end_price / start_price, 1 / max(total_days, 1)) - 1) / 390
    minute_volatility = daily_volatility / np.sqrt(390)

    # Loop through each day from the start date to the end date.
    while current_date <= end_date:
        # Check if the current day is a weekday (Monday=0, ..., Sunday=6).
        if current_date.weekday() < 5:
            # Set the starting time for the trading day.
            trading_time = datetime(current_date.year, current_date.month, current_date.day, 9, 30)
            
            # Loop 390 times to generate data for each minute of the trading day.
            for i in range(390):
                # Calculate the logarithmic return using the GBM formula.
                log_return = (minute_drift - (minute_volatility**2 / 2)) + (minute_volatility * np.random.normal(0, 1))
                # Calculate the new closing price based on the previous price and the log return.
                close_price = last_close_price * np.exp(log_return)
                # The opening price for this minute is the closing price of the last minute.
                open_price = last_close_price
                # Simulate the high and low prices for the minute.
                high_price = max(open_price, close_price) + abs(np.random.normal(0, minute_volatility * close_price * 0.5))
                low_price = min(open_price, close_price) - abs(np.random.normal(0, minute_volatility * close_price * 0.5))
                
                # Simulate a realistic "U-shaped" volume pattern (higher at open/close).
                time_frac = i / 389.0 if 389 > 0 else 0
                volume_multiplier = 1.0 + 1.5 * (1 - np.sin(np.pi * time_frac))
                volume = int(base_volume_per_min * volume_multiplier * (0.75 + np.random.uniform(0, 0.5)))
                
                # Append the newly created data point to our list.
                all_datapoints.append({
                    "o": round(open_price, 2), "h": round(high_price, 2), "l": round(low_price, 2),
                    "c": round(close_price, 2), "v": volume, "t": int(trading_time.timestamp() * 1000),
                    "vw": round((high_price + low_price + close_price) / 3, 4)
                })
                
                # Update the last closing price for the next iteration.
                last_close_price = close_price
                # Move to the next minute.
                trading_time += timedelta(minutes=1)
        
        # Move to the next day.
        current_date += timedelta(days=1)

    # If no data was generated (e.g., only weekends in the date range), exit early.
    if not all_datapoints:
        return "No trading days in the specified range. No data inserted."

    # ===================================================================
    # 3. Insert Data Using Optimized SQL
    #    This section takes the generated data and inserts it into the Snowflake table.
    # ===================================================================
    
    # Convert the list of Python dictionaries into a single JSON string.
    json_string = json.dumps(all_datapoints)
    # Escape any single quotes in the JSON string to prevent SQL errors.
    escaped_json_string = json_string.replace("'", "''")

    # Create the final SQL INSERT statement.
    sql = f"""
    INSERT INTO DEMODB.EQUITY_RESEARCH.DEMO_MARKET_DATA_JSON_INGESTION_TTBL (TICKER, RESULTS)
    SELECT
        TO_VARIANT('{ticker}'),
        PARSE_JSON('{escaped_json_string}')
    """
    
    # Execute the SQL statement.
    session.sql(sql).collect()
    # Return a success message with the total number of minutes processed.
    return f"Success! Processed and inserted {len(all_datapoints)} minutes of data for AAPL via Python procedure."
$$;

## Step 4: Check Existing Data Before Adding More

**Important**: Before calling the data generation procedure, always check what date ranges already exist to _**avoid overlapping data.**_


In [ ]:
-- Check existing data date ranges for AAPL
SELECT 
    TICKER::STRING as TICKER_SYMBOL,
    COUNT(*) as NUM_RECORDS,
    SUM(ARRAY_SIZE(RESULTS)) as TOTAL_DATA_POINTS,
    TO_DATE(TO_TIMESTAMP(MIN(RESULTS[0]:t::BIGINT) / 1000)) as FIRST_DATE,
    TO_DATE(TO_TIMESTAMP(MAX(RESULTS[ARRAY_SIZE(RESULTS)-1]:t::BIGINT) / 1000)) as LAST_DATE,
    DATEDIFF('day', 
        TO_DATE(TO_TIMESTAMP(MIN(RESULTS[0]:t::BIGINT) / 1000)),
        TO_DATE(TO_TIMESTAMP(MAX(RESULTS[ARRAY_SIZE(RESULTS)-1]:t::BIGINT) / 1000))
    ) + 1 as TOTAL_CALENDAR_DAYS
FROM DEMO_MARKET_DATA_JSON_INGESTION_TTBL 
WHERE TICKER::STRING = 'AAPL'
GROUP BY TICKER::STRING;


## Step 5: Generate Initial AAPL Historical Data

**⚠️ Date Range Guidelines:**
- If table is empty: Start with any date range (e.g., '2023-01-01' to '2023-12-31')
- If data exists from '2023-01-01' to '2023-12-31': Use '2024-01-01' to '2024-12-31' for next batch
- **Always avoid overlapping dates** to prevent duplicate data

Run this cell to generate your first batch of AAPL data:


In [ ]:
-- Generate initial AAPL data for 2023
-- ⚠️ IMPORTANT: Adjust dates based on existing data check above!
CALL DEMODB.EQUITY_RESEARCH.GENERATE_AAPL_DATA_SIMULATION_PY_FINAL('2023-01-01', '2023-12-31');


## Step 6: Create Dynamic Tables for VWAP Analysis

Now we'll create the Dynamic Tables that automatically process our raw data into VWAP calculations.


## Step 6.1: Create Trade Records Normalization Dynamic Table

This Dynamic Table flattens the JSON data into normalized individual trade records. It extracts each trade from the RESULTS array and converts timestamps to readable dates.


In [ ]:
-- Create trade records normalization dynamic table
CREATE OR REPLACE DYNAMIC TABLE DEMODB.EQUITY_RESEARCH.DEMO_TRADE_RECORDS_NORMALIZED_DTBL(
    TICKER_SYMBOL,
    TRADE_TIME,
    TRADE_OPEN,
    TRADE_HIGH,
    TRADE_LOW,
    TRADE_CLOSE,
    TRADE_VOLUME,
    TRADE_COUNT,
    TRADE_VWAP
)
TARGET_LAG = 'DOWNSTREAM'
REFRESH_MODE = INCREMENTAL
INITIALIZE = ON_CREATE
WAREHOUSE = DEMO_XSMALL_WH
COMMENT = 'Normalized trade records from JSON data - foundation for technical indicators'
AS
SELECT 
    raw_data.TICKER::STRING AS TICKER_SYMBOL,
    TO_TIMESTAMP(trade_record.value:t::BIGINT / 1000) AS TRADE_TIME,
    trade_record.value:o::FLOAT AS TRADE_OPEN,
    trade_record.value:h::FLOAT AS TRADE_HIGH,
    trade_record.value:l::FLOAT AS TRADE_LOW,
    trade_record.value:c::FLOAT AS TRADE_CLOSE,
    trade_record.value:v::INTEGER AS TRADE_VOLUME,
    trade_record.value:n::INTEGER AS TRADE_COUNT,
    trade_record.value:vw::FLOAT AS TRADE_VWAP
FROM DEMO_MARKET_DATA_JSON_INGESTION_TTBL raw_data,
     LATERAL FLATTEN(input => raw_data.RESULTS) trade_record
WHERE raw_data.TICKER::STRING = 'AAPL'  -- Focus on AAPL only
ORDER BY TICKER_SYMBOL, TRADE_TIME ASC;


## Step 6.2: Create VWAP 20-Minute Time Slices Dynamic Table

This Dynamic Table calculates VWAP values using 20-minute time slices. It aggregates trades within each time window and calculates the volume-weighted average price for intermediate analysis.


In [ ]:
-- Create 20-minute time slices dynamic table for intermediate VWAP
CREATE OR REPLACE DYNAMIC TABLE DEMODB.EQUITY_RESEARCH.DEMO_VWAP_20MIN_TIME_SLICES_DTBL(
    TRADE_TIME_SLICE,
    TICKER_SYMBOL,
    TICKER_SYMBOL_TRADE_TIME_SLICE,
    SUM_PRICE,
    SUM_VOLUME,
    INTERMEDIATE_SUM_PRICE_VOLUME,
    INTERMEDIATE_VWAP
)
TARGET_LAG = 'DOWNSTREAM'
REFRESH_MODE = INCREMENTAL
INITIALIZE = ON_CREATE
WAREHOUSE = DEMO_XSMALL_WH
COMMENT = '20-minute aggregated VWAP calculations for intermediate analysis'
AS
SELECT 
    TIME_SLICE(TRADE_TIME, 20, 'MINUTE') TRADE_TIME_SLICE,
    TICKER_SYMBOL,
    TICKER_SYMBOL || TO_VARCHAR(TIME_SLICE(TRADE_TIME, 20, 'MINUTE')) TICKER_SYMBOL_TRADE_TIME_SLICE,
    SUM(TRADE_CLOSE) SUM_PRICE,
    SUM(TRADE_VOLUME) SUM_VOLUME,
    SUM(TRADE_CLOSE * TRADE_VOLUME) INTERMEDIATE_SUM_PRICE_VOLUME,
    SUM(TRADE_CLOSE * TRADE_VOLUME) / SUM(TRADE_VOLUME) INTERMEDIATE_VWAP
FROM DEMO_TRADE_RECORDS_NORMALIZED_DTBL
GROUP BY TICKER_SYMBOL, TRADE_TIME_SLICE, TICKER_SYMBOL_TRADE_TIME_SLICE
ORDER BY TICKER_SYMBOL, TRADE_TIME_SLICE ASC;


## Step 6.3: Create VWAP Cumulative Analysis Dynamic Table

This Dynamic Table calculates the final cumulative VWAP using window functions. It maintains running totals of price×volume and volume to calculate the true VWAP over time for each stock.


In [ ]:
-- Create cumulative VWAP analysis dynamic table
CREATE OR REPLACE DYNAMIC TABLE DEMODB.EQUITY_RESEARCH.DEMO_VWAP_CUMULATIVE_ANALYSIS_DTBL(
    TRADE_TIME_SLICE,
    TICKER_SYMBOL,
    TICKER_SYMBOL_TRADE_TIME_SLICE,
    CUMULATIVE_PRICE,
    CUMULATIVE_VOLUME,
    FINAL_VWAP
)
TARGET_LAG = '8 hours'
REFRESH_MODE = INCREMENTAL
INITIALIZE = ON_CREATE
WAREHOUSE = DEMO_XSMALL_WH
COMMENT = 'Final cumulative VWAP calculations using window functions'
AS
SELECT 
    TRADE_TIME_SLICE,
    TICKER_SYMBOL,
    TICKER_SYMBOL_TRADE_TIME_SLICE,
    (SUM(SUM_PRICE) OVER (
        PARTITION BY TICKER_SYMBOL 
        ORDER BY TRADE_TIME_SLICE ASC 
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    )) CUMULATIVE_PRICE,
    (SUM(SUM_VOLUME) OVER (
        PARTITION BY TICKER_SYMBOL 
        ORDER BY TRADE_TIME_SLICE ASC 
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    )) CUMULATIVE_VOLUME,
    (SUM(INTERMEDIATE_SUM_PRICE_VOLUME) OVER (
        PARTITION BY TICKER_SYMBOL 
        ORDER BY TRADE_TIME_SLICE ASC 
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    )) / (SUM(SUM_VOLUME) OVER (
        PARTITION BY TICKER_SYMBOL 
        ORDER BY TRADE_TIME_SLICE ASC 
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    )) FINAL_VWAP
FROM DEMO_VWAP_20MIN_TIME_SLICES_DTBL
ORDER BY TICKER_SYMBOL, TRADE_TIME_SLICE ASC;


## Step 7: Understanding Dynamic Table Refresh

Dynamic Tables automatically refresh when their underlying data changes. Here's how to monitor and manually refresh them when needed:


In [ ]:
-- Check Dynamic Table basic information
SHOW DYNAMIC TABLES LIKE 'DEMO_%' IN SCHEMA DEMODB.EQUITY_RESEARCH;

-- View Dynamic Table configuration
SELECT 
    "name",
    "created_on",
    "target_lag",
    "refresh_mode",
    "warehouse",
    "comment"
FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()))
WHERE "name" LIKE 'DEMO_%'
ORDER BY "name";

-- Check Dynamic Table refresh history and status
SELECT 
    "NAME",
    "REFRESH_ACTION",
    "REFRESH_TRIGGER", 
    "STATE",
    "REFRESH_START_TIME",
    "REFRESH_END_TIME",
    "DATA_TIMESTAMP"
FROM TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY(
    name => 'DEMODB.EQUITY_RESEARCH.DEMO_TRADE_RECORDS_NORMALIZED_DTBL'
))
ORDER BY "REFRESH_END_TIME" DESC
LIMIT 5;


## Step 8: Manual Refresh of Dynamic Table

In [ ]:
-- Manual refresh of Dynamic Tables (if needed)
-- Run these commands to force refresh the Dynamic Tables in dependency order:

-- 1. First refresh the base normalization table
ALTER DYNAMIC TABLE DEMO_TRADE_RECORDS_NORMALIZED_DTBL REFRESH;

-- 2. Then refresh the 20-minute time slices table
ALTER DYNAMIC TABLE DEMO_VWAP_20MIN_TIME_SLICES_DTBL REFRESH;

-- 3. Finally refresh the cumulative analysis table
ALTER DYNAMIC TABLE DEMO_VWAP_CUMULATIVE_ANALYSIS_DTBL REFRESH;


## Step 9: Add More Historical Data (Optional)

To add more data, first check the existing date ranges, then call the procedure with non-overlapping dates:

**Example Scenarios:**
- **Current data**: 2023-01-01 to 2023-12-31 → **Add**: 2024-01-01 to 2024-12-31
- **Current data**: 2023-06-01 to 2023-12-31 → **Add**: 2023-01-01 to 2023-05-31 (backfill)
- **Empty table** → **Add**: Any date range you prefer
### After adding the data you can manually refresh the Dynamic Table. 
### Just refresh the downstream Dynamic Table. In this case it is DEMO_VWAP_CUMULATIVE_ANALYSIS_DTBL


In [ ]:
-- Add additional AAPL data (adjust dates based on existing data!)
-- ⚠️ CRITICAL: Check Step 4 results first to avoid overlapping dates!

-- Example: If you already have 2023 data, add 2024 data:
CALL DEMODB.EQUITY_RESEARCH.GENERATE_AAPL_DATA_SIMULATION_PY_FINAL('2024-01-01', '2024-12-31');

-- Example: If you need to backfill earlier data:
-- CALL DEMODB.EQUITY_RESEARCH.GENERATE_AAPL_DATA_SIMULATION_PY_FINAL('2022-01-01', '2022-12-31');

-- ⚠️ Uncomment and modify ONE of the above lines based on your data needs!


## Step 10: Verify VWAP Calculations

Compare intermediate vs final VWAP calculations and verify data quality.


In [ ]:
-- Compare intermediate vs final VWAP for AAPL
SELECT 
    i.TRADE_TIME_SLICE,
    i.TICKER_SYMBOL,
    ROUND(i.INTERMEDIATE_VWAP, 2) as INTERMEDIATE_VWAP,
    ROUND(f.FINAL_VWAP, 2) as FINAL_VWAP,
    ROUND(ABS(i.INTERMEDIATE_VWAP - f.FINAL_VWAP), 4) as VWAP_DIFFERENCE,
    i.SUM_VOLUME as PERIOD_VOLUME,
    f.CUMULATIVE_VOLUME
FROM DEMO_VWAP_20MIN_TIME_SLICES_DTBL i
JOIN DEMO_VWAP_CUMULATIVE_ANALYSIS_DTBL f
    ON i.TICKER_SYMBOL_TRADE_TIME_SLICE = f.TICKER_SYMBOL_TRADE_TIME_SLICE
WHERE i.TICKER_SYMBOL = 'AAPL'
ORDER BY i.TRADE_TIME_SLICE ASC
LIMIT 20;


## Step 11: Interactive Visualization

Create an interactive Streamlit dashboard.


## Visualize the 20-Minute Time Slice Volume Weighted Average Price & the Cumulative Volume-Weighted Average Price.  

In [ ]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
session = get_active_session()
# Get list of available stocks
stocks_query = """
SELECT DISTINCT TICKER_SYMBOL 
FROM DEMO_VWAP_CUMULATIVE_ANALYSIS_DTBL 
ORDER BY TICKER_SYMBOL
"""
stocks_df = session.sql(stocks_query).to_pandas()
available_stocks = stocks_df['TICKER_SYMBOL'].tolist()

# Streamlit UI
st.title('📈 Stock VWAP Analysis Dashboard')
st.markdown('### Volume Weighted Average Price (VWAP) Visualization')

# Stock selector
selected_stock = st.selectbox(
    'Select a Stock:', 
    available_stocks,
    index=0
)

# Query data for selected stock
vwap_query = f"""
SELECT 
    i.TRADE_TIME_SLICE,
    i.TICKER_SYMBOL,
    i.SUM_VOLUME,
    i.INTERMEDIATE_VWAP,
    f.CUMULATIVE_VOLUME,
    f.FINAL_VWAP
FROM DEMO_VWAP_20MIN_TIME_SLICES_DTBL i
JOIN DEMO_VWAP_CUMULATIVE_ANALYSIS_DTBL f 
    ON i.TICKER_SYMBOL = f.TICKER_SYMBOL 
    AND i.TRADE_TIME_SLICE = f.TRADE_TIME_SLICE
WHERE i.TICKER_SYMBOL = '{selected_stock}'
ORDER BY i.TRADE_TIME_SLICE
"""

df = session.sql(vwap_query).to_pandas()

if not df.empty:
    # Create subplot with secondary y-axis
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('VWAP Comparison', 'Volume Analysis'),
        specs=[[{"secondary_y": False}],
               [{"secondary_y": True}]],
        vertical_spacing=0.1
    )
    
    # Add VWAP lines to first subplot
    fig.add_trace(
        go.Scatter(
            x=df['TRADE_TIME_SLICE'],
            y=df['INTERMEDIATE_VWAP'],
            mode='lines+markers',
            name='Intermediate VWAP',
            line=dict(color='blue', width=2),
            marker=dict(size=6)
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df['TRADE_TIME_SLICE'],
            y=df['FINAL_VWAP'],
            mode='lines+markers',
            name='Final VWAP',
            line=dict(color='red', width=2),
            marker=dict(size=6)
        ),
        row=1, col=1
    )
    
    # Add volume bars to second subplot
    fig.add_trace(
        go.Bar(
            x=df['TRADE_TIME_SLICE'],
            y=df['SUM_VOLUME'],
            name='Period Volume',
            marker_color='lightblue',
            opacity=0.7
        ),
        row=2, col=1
    )
    
    # Add cumulative volume line to second subplot
    fig.add_trace(
        go.Scatter(
            x=df['TRADE_TIME_SLICE'],
            y=df['CUMULATIVE_VOLUME'],
            mode='lines+markers',
            name='Cumulative Volume',
            line=dict(color='green', width=2),
            marker=dict(size=4),
            yaxis='y4'
        ),
        row=2, col=1
    )
    
    # Update layout
    fig.update_layout(
        title=f'{selected_stock} - VWAP Analysis',
        height=600,
        showlegend=True,
        hovermode='x unified'
    )
    
    # Update y-axis labels
    fig.update_yaxes(title_text="VWAP Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Period Volume", row=2, col=1)
    fig.update_yaxes(title_text="Cumulative Volume", secondary_y=True, row=2, col=1)
    
    # Update x-axis labels
    fig.update_xaxes(title_text="Time Slice", row=2, col=1)
    
    st.plotly_chart(fig, use_container_width=True)
    
    # Display summary statistics
    st.subheader(f'{selected_stock} VWAP Summary')
    
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        st.metric(
            "Final VWAP", 
            f"${df['FINAL_VWAP'].iloc[-1]:.4f}"
        )
    
    with col2:
        st.metric(
            "Total Volume", 
            f"{df['CUMULATIVE_VOLUME'].iloc[-1]:,}"
        )
    
    with col3:
        st.metric(
            "Price Range", 
            f"${df['FINAL_VWAP'].min():.2f} - ${df['FINAL_VWAP'].max():.2f}"
        )
    
    with col4:
        st.metric(
            "Time Periods", 
            f"{len(df)}"
        )
    
    # Display raw data table
    with st.expander("View Raw Data"):
        st.dataframe(
            df.round(4),
            use_container_width=True
        )
        
else:
    st.error(f"No data found for {selected_stock}")
